In [1]:
import torch
import syft as sy
import pandas as pd
from syft.lib.python.list import List
print(sy.__version__)
print(torch.__version__)

0.5.0
1.8.1+cpu


In [27]:
import numpy as np
import torch as th
import pandas as pd
from PIL import Image
import imageio.v2 as imageio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset,DataLoader

In [3]:
df =pd.read_csv('HAM10000_metadata.csv')

In [4]:
target = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [5]:
subset_metadata_a = df.sample(frac=0.2, random_state=42) 

In [6]:
num_samples_per_label_a = 200                                                 ## 200 sample for each class
balanced_sub_a = pd.DataFrame()
for label in target:
    label_metadata_a = subset_metadata_a[subset_metadata_a['dx'] == label]
    label_samples_a = label_metadata_a.sample(n=num_samples_per_label_a, random_state=42, replace = True)
    balanced_sub_a = pd.concat([balanced_sub_a, label_samples_a])

In [7]:
size = (64, 64)
images_sub_a = []
for i, row in balanced_sub_a.iterrows():
    img_a = imageio.imread('HAM_data//HAM10000/' + row['image_id'] + '.jpg')
    resized_img = np.array(Image.fromarray(img_a).resize(size))
    images_sub_a.append(resized_img)

In [8]:
images_and_labels_sub_a = []
for i, row in balanced_sub_a.iterrows():
    img_b = imageio.imread('HAM_data/HAM10000/' + row['image_id'] + '.jpg')
    resized_img = np.array(Image.fromarray(img_b).resize(size))
    label = row['dx']
    images_and_labels_sub_a.append((resized_img, label))

In [9]:
images_sub_a, labels_sub_a = zip(*images_and_labels_sub_a)
images_sub_a = np.array(images_sub_a)
labels_sub_a = np.array(labels_sub_a)

# Print the shapes of the resulting arrays
print('Image shape:', images_sub_a.shape)
print('Label shape:', labels_sub_a.shape)

Image shape: (1400, 64, 64, 3)
Label shape: (1400,)


In [10]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_sub_a)

In [14]:
image_tensor = th.FloatTensor(images_sub_a.astype('float32'))
label_tensor = th.FloatTensor(labels_encoded.astype('float32'))

In [15]:
import torchvision.transforms as transforms
from PIL import Image
import torch

transform_train = transforms.Compose([
    transforms.RandomCrop(64, padding=0),    
    transforms.RandomHorizontalFlip(),    
    transforms.ToTensor(),    
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Apply the transformations to the data
x_train_transformed = []
for img in images_sub_a:
    img_pil = Image.fromarray(img)
    img_transformed = transform_train(img_pil)
    x_train_transformed.append(img_transformed)

In [16]:
x_train_transformed = torch.stack(x_train_transformed)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_train_transformed, label_tensor, train_size=0.7, random_state=2019)

In [28]:
train_dataset = TensorDataset(x_train, y_train)
test_dataset= TensorDataset(x_test, y_test)

In [29]:
batch_size = 1400
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader= DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [18]:
server1 = sy.VirtualMachine(name="server1")
client1 = server1.get_root_client()
remote_torch = client1.torch

In [19]:
client1.store.pandas

""


In [20]:
class LeNet(sy.Module):
    def __init__(self, torch_ref):
        super(LeNet, self).__init__(torch_ref=torch_ref)
        self.conv1 = torch_ref.nn.Conv2d(3, 6, kernel_size=5)
        self.pool1 = torch_ref.nn.MaxPool2d(kernel_size=2)
        self.conv2 = torch_ref.nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = torch_ref.nn.MaxPool2d(kernel_size=2)
        self.relu = torch_ref.nn.ReLU()
        self.flat = torch_ref.nn.Flatten(start_dim=1)
        self.fc1 = torch_ref.nn.Linear(16*13*13, 120)
        self.fc2 = torch_ref.nn.Linear(120, 84)
        self.fc3 = torch_ref.nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.flat(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
local_model = LeNet(torch)

In [22]:
client1_model = local_model.send(client1)

In [23]:
client1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 60996ba46a354e408c295e1d1c39d703>,[],,<class 'torch.nn.modules.module.Module'>
1,<UID: 366e48e540874ca98196a09375221c20>,[],,<class 'torch.nn.modules.conv.Conv2d'>
2,<UID: e7d8b4c090364f578cb2c299697f3072>,[],,<class 'torch.nn.modules.pooling.MaxPool2d'>
3,<UID: ac8d6784690446cf867857662afe0337>,[],,<class 'torch.nn.modules.conv.Conv2d'>
4,<UID: 96fab7d2a9de4f53b1394bad2d0e06d7>,[],,<class 'torch.nn.modules.pooling.MaxPool2d'>
5,<UID: 9a311b9430da4f89b24625a99cc2476c>,[],,<class 'torch.nn.modules.activation.ReLU'>
6,<UID: 30bdd684e60f49bd9728626faa3545ce>,[],,<class 'torch.nn.modules.flatten.Flatten'>
7,<UID: e635102bde08493f8e4e6aee26074fed>,[],,<class 'torch.nn.modules.linear.Linear'>
8,<UID: 8491313f013f4cffbaff2984ed23dbfd>,[],,<class 'torch.nn.modules.linear.Linear'>
9,<UID: a2ff5da03f6f45fc85f6cd336f796184>,[],,<class 'torch.nn.modules.linear.Linear'>


In [24]:
opt1 = remote_torch.optim.Adam(client1_model.parameters(), lr=0.01)

In [25]:
client1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 60996ba46a354e408c295e1d1c39d703>,[],,<class 'torch.nn.modules.module.Module'>
1,<UID: 366e48e540874ca98196a09375221c20>,[],,<class 'torch.nn.modules.conv.Conv2d'>
2,<UID: e7d8b4c090364f578cb2c299697f3072>,[],,<class 'torch.nn.modules.pooling.MaxPool2d'>
3,<UID: ac8d6784690446cf867857662afe0337>,[],,<class 'torch.nn.modules.conv.Conv2d'>
4,<UID: 96fab7d2a9de4f53b1394bad2d0e06d7>,[],,<class 'torch.nn.modules.pooling.MaxPool2d'>
5,<UID: 9a311b9430da4f89b24625a99cc2476c>,[],,<class 'torch.nn.modules.activation.ReLU'>
6,<UID: 30bdd684e60f49bd9728626faa3545ce>,[],,<class 'torch.nn.modules.flatten.Flatten'>
7,<UID: e635102bde08493f8e4e6aee26074fed>,[],,<class 'torch.nn.modules.linear.Linear'>
8,<UID: 8491313f013f4cffbaff2984ed23dbfd>,[],,<class 'torch.nn.modules.linear.Linear'>
9,<UID: a2ff5da03f6f45fc85f6cd336f796184>,[],,<class 'torch.nn.modules.linear.Linear'>


In [31]:
for epoch in range(5):
    for i, batch in enumerate(train_dataloader):
        image, label = batch

        # Send data to the remote worker
        image_ptr = image.send(client1)
        label_ptr = label.send(client1)

        # Perform forward pass on the remote model
        output_ptr = client1_model(image_ptr)
        output_ptr = output_ptr.detach()  # Detach the tensor

        # Compute loss on the remote worker
        loss = remote_torch.nn.CrossEntropyLoss()(output_ptr, label_ptr.long())

        # Retrieve the loss back to the local worker
        loss_val = loss.get()

        # Perform backward pass on the local model
        loss.backward()

        # Update model parameters on the remote worker
        opt1.step()

        # Clear the gradients on the local model
        opt1.zero_grad()

        # Print the loss
    print(f"epoch: {epoch} loss: {loss_val}")

[2023-05-07T12:26:02.464403-0600][CRITICAL][logger]][27316] execute_action on torch.Tensor.backward failed due to missing object at: <UID: 1522888ebd7f4e7fb7e5d23e3099803d>
[2023-05-07T12:26:02.464403-0600][CRITICAL][logger]][27316] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 197ed514df0c4e76acb759e317707581>.


epoch: 0 loss: 2.294703483581543


[2023-05-07T12:26:03.523207-0600][CRITICAL][logger]][27316] execute_action on torch.Tensor.backward failed due to missing object at: <UID: fa71de88b50e4469b558d56958d8e5ce>
[2023-05-07T12:26:03.523207-0600][CRITICAL][logger]][27316] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 2056ed175e7d4a80a0cb261db1e88d84>.


epoch: 1 loss: 2.294703722000122


[2023-05-07T12:26:04.510024-0600][CRITICAL][logger]][27316] execute_action on torch.Tensor.backward failed due to missing object at: <UID: 058b431b529f4639871cfa04b54582f2>
[2023-05-07T12:26:04.510024-0600][CRITICAL][logger]][27316] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: e217e06c60524a9199b49dac5bfce966>.


epoch: 2 loss: 2.294701099395752


[2023-05-07T12:26:05.488491-0600][CRITICAL][logger]][27316] execute_action on torch.Tensor.backward failed due to missing object at: <UID: 5f4598d08ecc448dbf2decf742093255>
[2023-05-07T12:26:05.491146-0600][CRITICAL][logger]][27316] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 59561c37e0714e6db28ed7f09a37abea>.


epoch: 3 loss: 2.294701337814331


[2023-05-07T12:26:06.462585-0600][CRITICAL][logger]][27316] execute_action on torch.Tensor.backward failed due to missing object at: <UID: 51b99cccdd0944849463b424e01650fe>
[2023-05-07T12:26:06.462585-0600][CRITICAL][logger]][27316] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 8eef9b4289a7403491aa72ba65992b27>.


epoch: 4 loss: 2.2947044372558594
